<a href="https://colab.research.google.com/github/Danielmr1/Tesis/blob/master/quemas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# BURNED AREA DETECTION - Google Earth Engine en Google Colab
# PARTE 1: INSTALACIÓN, COMPATIBILIDAD Y AUTENTICACIÓN
# ============================================================================

# Forzar versión compatible de google-auth para evitar warning por dependencias
!pip install google-auth==2.38.0 --quiet

# Reinicia manualmente el runtime después de ejecutar este bloque para que tenga efecto.

# Instalación de librerías necesarias (opcional si ya las tienes)
!pip install --upgrade earthengine-api -q
!pip install rasterio rioxarray folium geemap -q

# Importar librerías y obtener autenticación
import ee
from google.colab import auth
auth.authenticate_user()
ee.Initialize(project='generar-contorno')

print("✓ Autenticación completada")

# Importar otras librerías útiles
import os, json, numpy as np, pandas as pd
from datetime import datetime, timedelta
from IPython.display import Image, display
import rasterio
import warnings
warnings.filterwarnings('ignore')
import folium
from google.colab import files, drive
import glob
import time



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00
✓ Autenticación completada


In [3]:
# ============================================================================
# PARTE 2: CARGAR DATOS Y DEFINIR FUNCIONES
# ============================================================================

fireAreas = ee.FeatureCollection('projects/generar-contorno/assets/FIRMS_2020')


#Cambiar confidence de acuerdo al caso,
filterConfidenceH = fireAreas.filter(ee.Filter.eq('confidence', 'h'))
filterConfidenceNum = fireAreas.filter(ee.Filter.gte('confidence', 70))
filteredFireAreas = filterConfidenceH.merge(filterConfidenceNum)


print(f"✓ Áreas filtradas: {filteredFireAreas.size().getInfo()}")





def maskClouds(image):
    qa = image.select('QA60')
    cloudBitMask = ee.Number(1).leftShift(10)
    cirrusBitMask = ee.Number(1).leftShift(11)
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

def calculateDeltaNBR(feature):
    acqDate = ee.Date(feature.get('acq_date'))
    pre_fire_start = acqDate.advance(-60, 'day')
    pre_fire_end = acqDate.advance(-1, 'day')
    post_fire_start = acqDate.advance(1, 'day')
    post_fire_end = acqDate.advance(60, 'day')

    pre_fire_image = (ee.ImageCollection('COPERNICUS/S2')
                     .filterBounds(feature.geometry())
                     .filterDate(pre_fire_start, pre_fire_end)
                     .map(maskClouds).median())

    post_fire_image = (ee.ImageCollection('COPERNICUS/S2')
                      .filterBounds(feature.geometry())
                      .filterDate(post_fire_start, post_fire_end)
                      .map(maskClouds).median())

    nbr_pre = pre_fire_image.normalizedDifference(['B8', 'B12']).rename('NBR_pre')
    nbr_post = post_fire_image.normalizedDifference(['B8', 'B12']).rename('NBR_post')
    delta_nbr = nbr_pre.subtract(nbr_post).rename('dNBR')

    # Cambiar threshold según el caso
    #threshold = 0.27
    threshold = 0.27
    burned_area_mask_int16 = delta_nbr.gt(threshold).toInt16().clip(feature.geometry())

    baseDate = ee.Date('1970-01-01')
    date_band = ee.Image.constant(acqDate.difference(baseDate, 'day')).rename('burn_date').toInt16()
    date_band_masked = date_band.updateMask(burned_area_mask_int16)

    burned_mask_with_date = burned_area_mask_int16.addBands(date_band_masked)
    return burned_mask_with_date.set({'system:index': feature.get('system:index'), 'acq_date': feature.get('acq_date')})

print("✓ Funciones definidas")


✓ Áreas filtradas: 228
✓ Funciones definidas


In [23]:
# ============================================================================
# PARTE 3: PROCESAR Y EXPORTAR
# ============================================================================

print("\n" + "="*70)
print("PROCESANDO TODAS LAS ÁREAS")
print("="*70)

fireAreasList = filteredFireAreas.toList(filteredFireAreas.size())
totalAreas = filteredFireAreas.size().getInfo()
print(f"Total de áreas: {totalAreas}")

allBurnedMasks = []
for index in range(totalAreas):
    feature = ee.Feature(fireAreasList.get(index))
    burned_mask = calculateDeltaNBR(feature)
    allBurnedMasks.append(burned_mask)
    if (index + 1) % 10 == 0:
        print(f"  Procesadas {index + 1}/{totalAreas}")

allBurnedCollection = ee.ImageCollection.fromImages(allBurnedMasks)
allBurnedMosaic = allBurnedCollection.mosaic()
print(f"✓ Procesadas todas las {totalAreas} áreas")

targetCRS = 'EPSG:32718'
scale = 20
allBurnedMosaic_utm = allBurnedMosaic.reproject(crs=targetCRS, scale=scale)
print(f"✓ Reproyectado a {targetCRS}")

print("\n" + "="*70)
print("EXPORTANDO A GOOGLE DRIVE")
print("="*70)

task1 = ee.batch.Export.image.toDrive(
    image=allBurnedMosaic_utm,
    description='burned_areas_all_filtered_FIRMS_UTM',
    scale=scale,
    region=filteredFireAreas.geometry(),
    fileFormat='GeoTIFF',
    maxPixels=1e13
)
task1.start()
print("✓ Exportación iniciada")



PROCESANDO TODAS LAS ÁREAS
Total de áreas: 228
  Procesadas 10/228
  Procesadas 20/228
  Procesadas 30/228
  Procesadas 40/228
  Procesadas 50/228
  Procesadas 60/228
  Procesadas 70/228
  Procesadas 80/228
  Procesadas 90/228
  Procesadas 100/228
  Procesadas 110/228
  Procesadas 120/228
  Procesadas 130/228
  Procesadas 140/228
  Procesadas 150/228
  Procesadas 160/228
  Procesadas 170/228
  Procesadas 180/228
  Procesadas 190/228
  Procesadas 200/228
  Procesadas 210/228
  Procesadas 220/228
✓ Procesadas todas las 228 áreas
✓ Reproyectado a EPSG:32718

EXPORTANDO A GOOGLE DRIVE
✓ Exportación iniciada


In [ ]:
import time
import glob
import os
from google.colab import drive, files

# ========================================================================
# PARTE 4: ESPERAR Y DESCARGAR
# ========================================================================

print("\nEsperando exportación (máximo 15 minutos)...")
max_wait = 900  # 15 minutos
elapsed = 0
while elapsed < max_wait:
    status = task1.status().get('state', 'UNKNOWN')
    print(f"  [{elapsed}s] Estado: {status}")
    if status == 'COMPLETED':
        print(f"✓ Completada en {elapsed}s")
        break
    elif status == 'FAILED':
        error_msg = task1.status().get('error_message', 'Sin detalles')
        print(f"✗ Falló: {error_msg}")
        break
    time.sleep(30)
    elapsed += 30
else:
    print("✗ Tiempo máximo de espera excedido. Exportación no completada.")

print("\n" + "="*70)
print("MONTAJE DE GOOGLE DRIVE")

# Intenta desmontar primero por si está montado
try:
    drive.flush_and_unmount()
    print("✓ Drive desmontado correctamente")
except Exception as e:
    print(f"⚠ No se pudo desmontar drive (posible que no esté montado): {e}")

# Monta drive sin usar force_remount para minimizar fallos
drive.mount('/content/gdrive')
print("✓ Drive montado correctamente")

print("\nBuscando archivos GeoTIFF con patrón 'FIRMS*UTM' en Google Drive...")

# Busca archivos recursivamente en la carpeta de Drive montada
geotiff_files = glob.glob('/content/gdrive/My Drive/**/*FIRMS*UTM*.tif', recursive=True)

if geotiff_files:
    print(f"✓ {len(geotiff_files)} archivo(s) encontrados:")
    for filepath in geotiff_files:
        try:
            size_mb = os.path.getsize(filepath) / (1024 * 1024)
            print(f"  Descargando {os.path.basename(filepath)} ({size_mb:.1f} MB)...")
            files.download(filepath)
        except Exception as e:
            print(f"  ✗ Error al descargar {os.path.basename(filepath)}: {e}")
    print("✓ Descargas completadas")
else:
    print("✗ No se encontraron archivos")

print("\n" + "="*70)
print("¡PROCESO COMPLETADO!")
print("="*70)
